In [ ]:
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import seaborn
import glob
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '../') # path to functions
import cvasl.seperated as sep
from cvasl.file_handler import Config

## Read datasets into pandas dataframes

We have these in our open_work

In [ ]:
# Identify files
datasets_folder = 'C:/Projects/brainspin/not_pushed/data_anonymized/assembled'
dataset_files = glob.glob(os.path.join(datasets_folder, '*.csv'))
print(dataset_files)

In [ ]:
!pwd

In [ ]:
# Read files into dataframes
TOP = pd.read_csv('C:/Projects/brainspin/not_pushed/data_anonymized/assembled\\top_stitched.csv')
StrokeMRI = pd.read_csv('C:/Projects/brainspin/not_pushed/data_anonymized/assembled\\StrokeMRI_stitched.csv')
#Insight46 = pd.read_csv('../open_work/internal_results\\Insight46_stitched.csv')
df_list = [TOP, StrokeMRI] #, Insight46]


In [ ]:
TOP = TOP.drop(0, axis=0)
StrokeMRI = StrokeMRI.drop(0, axis=0)
#Insight46 = Insight46.drop(0, axis=0)


In [ ]:
# Insight46_described = Insight46.describe()
# Insight46_described

In [ ]:
StrokeMRI_described = StrokeMRI.describe()
StrokeMRI_described

TOP was produced in a way that made more data non-numeric do extra step:

In [ ]:
TOP =  TOP.apply (pd.to_numeric, errors='coerce')
TOP_described =TOP.describe()
TOP_described

In [ ]:
common_columns = TOP.columns.intersection(StrokeMRI.columns).to_list()

## Now we blend the two sets

In [ ]:
super_set = pd.concat((TOP[common_columns],StrokeMRI[common_columns]))
super_set

In [ ]:
idx = len(super_set) - 1 if len(super_set) % 2 else len(super_set)

In [ ]:
super_set_sorted = super_set.sort_values('Age')#apply(lambda x: x.interpolate())
#print(len(super_set_sorted))
super_set_f= pd.DataFrame(super_set_sorted.drop('session', axis=1))
super_set_f = super_set_f.dropna()
super_set_f

In [ ]:
super_set_f = super_set_f.reset_index()
super_set_f

In [ ]:
# print(type(super_set_f))
new_set = super_set_f[:idx].groupby(super_set_f.index[:idx] // 2).mean()
#super_set_f

So we took our two datasets and blended by simply mixing close values i.e. every adjacent on one side row in a cobined dataset, when sorted by age. Now we can add a bit of noise just for good measure. 

In [ ]:
mu, sigma = 0, 0.01 
# creating a noise with the same dimension as the dataset 
noise = np.random.normal(mu, sigma, [527,25]) 
print(noise)

Add the two

In [ ]:
mixed_and_noise = noise+ new_set
mixed_and_noise 

## OK, so let's look at this in terms of the way it graphs

In [ ]:
sep.relate_columns_graphs(mixed_and_noise ,'Age')

Everything looks fine except sex, we need to recode that as sex will be for us binary

In [ ]:
mixed_and_noise= mixed_and_noise.drop('index', axis=1)
# as we added noise, we can just round and denoise
mixed_and_noise['Sex'] = mixed_and_noise['Sex'].round()
mixed_and_noise

In [ ]:
## We are done. This gives us a synthetic/augmented dataset based upon our existing ones. Pending approval

In [ ]:
mixed_and_noise.to_csv('C:/Projects/brainspin/not_pushed/synthetic_set.csv')